## Importing libs

In [151]:
import time
import numpy as np
import pandas as pd
import pickle

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import json
import dateparser

In [152]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('headless')    
service = Service('./chromedriver')
driver = webdriver.Chrome(service=service, options=options)

## Functions

In [161]:
def get_content(driver: webdriver, api: str) -> json:
    '''Open a link by selenium webdriver, scrap the page and returns result with proposed news in json format'''
    driver.get(api) # открываем ссылку
    time.sleep(3) # даем прогрузиться
    
    page_source = driver.page_source # берем страницу
    soup = BeautifulSoup(page_source, 'html.parser') # парсим
    data = json.loads(soup.body.text) # достаем текст и превращаем в формат json
    
    return data['html']

def update_news(news: list, data: BeautifulSoup, used_ids_path=None):
    # если вдргу файла нет, то содаст пустой список
    if used_ids_path:
        used_ids = pd.read_csv(used_ids_path)['news_id'].tolist()
    else:
        used_ids = []
        
    soup = BeautifulSoup(data, 'html.parser')
    content_divs = soup.find_all('div', class_='item__wrap l-col-center') # в этом классе лежит, все что нам нужно
    
    for item in content_divs:
        item_link = item.find('a', class_="item__link")['href'] # ссылка на новость
        item_id = item_link.split('/')[-1] # свопадает с id новости
        if item_id in used_ids: # скипаем, если вдруг новость уже была добавлена
            continue
        cur_news = {}
        cur_news['published_date'] = dateparser.parse(item.find('span', class_="item__category").text) # парсим дату
        cur_news['news_id'] = item_id # добавляем id новости
        cur_news['link'] = item_link # ссылка на новость
        cur_news['title'] = item.find('span', class_="item__title rm-cm-item-text").text.strip() # текст
            
        used_ids.append(cur_news['news_id'])
        news.append(cur_news)
    
    pd.Series(used_ids).to_csv('used_ids_rbc.csv', header=['news_id'], index=False) # сохраняем новый список
    pass   

## Parsing

In [175]:
api = 'https://perm.rbc.ru/v10/ajax/get-news-by-filters/?region=perm&offset=0&limit=20'
news = []
data = get_content(driver, api)
update_news(news, data)

In [176]:
news

[{'published_date': datetime.datetime(2022, 6, 21, 11, 11),
  'news_id': '62b17cfb9a79479997f2f571',
  'link': 'https://perm.rbc.ru/perm/freenews/62b17cfb9a79479997f2f571',
  'title': 'В Прикамье продолжает снижаться суточная заболеваемость коронавирусом'},
 {'published_date': datetime.datetime(2022, 6, 21, 11, 8),
  'news_id': '62b17c1a9a794799866008b7',
  'link': 'https://perm.rbc.ru/perm/freenews/62b17c1a9a794799866008b7',
  'title': 'На заседание оргкомитета «Пермь-300» приедет вице-премьер Чернышенко'},
 {'published_date': datetime.datetime(2022, 6, 21, 9, 45),
  'news_id': '62b168b09a79479027cf2f7d',
  'link': 'https://perm.rbc.ru/perm/freenews/62b168b09a79479027cf2f7d',
  'title': 'Строительство трамвайных веток на Парковый и Садовый перенесли на 2023 г.'},
 {'published_date': datetime.datetime(2022, 6, 21, 8, 37),
  'news_id': '62b158c59a794789d921b664',
  'link': 'https://perm.rbc.ru/perm/freenews/62b158c59a794789d921b664',
  'title': 'Объем федерального инфраструктурного кред

In [150]:
driver.close()

In [137]:
df['news_id'].to_csv('used_ids_rbc.csv', index=False)

In [ ]:
regions = ['ekb', 'perm']

In [177]:
i = 0
while True:
    i += 20
    time.sleep(2)
    iter_api = f'https://perm.rbc.ru/v10/ajax/get-news-by-filters/?region=perm&offset={i}&limit=20'
    data = get_content(driver, iter_api)
    update_news(news, data, './used_ids_rbc.csv')
    if i % 200 == 0:
        with open('rbc_perm', 'wb') as f:
            pickle.dump(news, f)

JSONDecodeError: Expecting value: line 6 column 13 (char 17)

In [172]:
len(news)

1020

In [178]:
with open('rbc_perm', 'wb') as f:
    pickle.dump(news, f)

In [174]:
1020 / 20

51.0

In [183]:
news[-1]

{'published_date': datetime.datetime(2022, 2, 28, 10, 27),
 'news_id': '621c782d9a79474909668f06',
 'link': 'https://perm.rbc.ru/perm/freenews/621c782d9a79474909668f06',
 'title': 'Депутат Госдумы Дмитрий Скриванов также попал под санкции ЕС'}